In [62]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np

import nltk

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler

from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import GridSearchCV

#### Sistema de recomendación item-item:

- **def recomendacion_juego( item_id ):**

> Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.


In [2]:
# Ruta del archivo .parquet
file_path = r'datasets\processed_data\steam_games.parquet'

# Leer el archivo .parquet en un DataFrame
df = pq.read_table(file_path).to_pandas()


In [3]:
# Nos quedamos con las columnas de interes
df = df[['item_id', 'genres']]
# quitamos duplicados y reiniciamos el índice
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.shape


(22527, 2)

In [4]:
# Importamos nuestro modelo
tf = TfidfVectorizer()

# calculamos los features (en genres) para cada ítem_id 
tfidf_matrix = tf.fit_transform(df['genres'])

In [5]:
# calculamos las similitudes entre todos los documentos
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:
def recommend_items(item_id, cosine_similarities, items, n=5):
  # Obtener el índice del elemento actual
  index = np.where(items == item_id)[0][0]

  # Obtener las similitudes del elemento actual con todos los demás elementos
  similar_items = list(enumerate(cosine_similarities[index]))

  # Ordenar las similitudes en orden descendente
  similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)[1:n+1]

  # Obtener los índices de los elementos más similares
  item_indices = [i[0] for i in similar_items]

  # Devolver los IDs de los elementos más similares
  return items[item_indices]

  # Obtener recomendaciones para un elemento específico
item_id = '252490'
recommendations = recommend_items(item_id, cosine_similarities, df['item_id'])

# Imprimir las recomendaciones
print('Recommendations for item {}:'.format(item_id))
for item in recommendations:
  print(item)

In [7]:
def recommend_items(item_id, cosine_similarities, items, n=5):
    # Obtener el índice del elemento actual
    index = np.where(items == item_id)[0][0]

    # Obtener las similitudes del elemento actual con todos los demás elementos
    similar_items = list(enumerate(cosine_similarities[index]))

    # Ordenar las similitudes en orden descendente
    similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)[1:n+1]

    # Obtener los índices de los elementos más similares
    item_indices = [i[0] for i in similar_items]

    # Devolver los IDs de los elementos más similares
    return items[item_indices]

# Lista para almacenar las predicciones
all_predictions = []

# Iterar sobre todos los item_id
for item_id in df['item_id'].unique():
    # Obtener las recomendaciones para el item_id actual
    recommendations = recommend_items(item_id, cosine_similarities, df['item_id'])
    # Guardar los resultados en la lista
    all_predictions.append({"item_id": item_id, "top_5_item_id": recommendations})

# Convertir la lista de diccionarios en un DataFrame
predictions_df = pd.DataFrame(all_predictions)



      item_id                                      top_5_item_id
0      761140  2740    350810
2882    361520
4151    262120
5...
1      643980  2846    365270
3798    344190
4587    465830
4...
2      670290  5357     522660
10094    729580
4813     47803...
3      767400  30      764280
366      19080
512      60340
1...
4      772540  1761    289480
1993    297100
2333    318220
2...
...       ...                                                ...
22522  745400  1458    244810
1561    257750
1882    238460
1...
22523  773640  678     901776
680      80202
681      80204
6...
22524  733530  791    206690
802    207710
846    108232
859 ...
22525  610660  6023     575400
6288     361840
8944     76018...
22526  658870  36      4900
37      2520
95     13000
147    ...

[22527 rows x 2 columns]


In [10]:
def recommend_items(item_id, cosine_similarities, items, n=5):
    # Obtener el índice del elemento actual
    index = np.where(items == item_id)[0][0]

    # Obtener las similitudes del elemento actual con todos los demás elementos
    similar_items = list(enumerate(cosine_similarities[index]))

    # Ordenar las similitudes en orden descendente
    similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)[1:n+1]

    # Obtener los índices de los elementos más similares
    item_indices = [i[0] for i in similar_items]

    # Devolver los IDs de los elementos más similares
    return items[item_indices]

# Lista para almacenar las predicciones
all_predictions = []

# Iterar sobre todos los item_id
for item_id in df['item_id'].unique():
    # Obtener las recomendaciones para el item_id actual
    recommendations = recommend_items(item_id, cosine_similarities, df['item_id'])
    # Guardar los resultados en la lista
    all_predictions.append({"item_id": item_id, "top_5_item_id": ' '.join(map(str, recommendations))})


      item_id                       top_5_item_id
0      761140  350810 361520 262120 504880 592730
1      643980  365270 344190 465830 465831 465832
2      670290  522660 729580 478034 486190 485143
3      767400    764280 19080 60340 227380 237630
4      772540  289480 297100 318220 364800 364810
...       ...                                 ...
22522  745400  244810 257750 238460 277510 297370
22523  773640     901776 80202 80204 80207 238930
22524  733530  206690 207710 108232 108230 108200
22525  610660  575400 361840 760180 509130 507070
22526  658870         4900 2520 13000 21300 18300

[22527 rows x 2 columns]


In [14]:
# Exportamos para trabajar los dataos resultantes de las predicciones
item_to_item = pd.DataFrame(all_predictions)
item_to_item.to_parquet('datasets/processed_data/item_to_item.parquet')


In [82]:
# Ruta al archivo Parquet
file_path = 'datasets/processed_data/data_for_ML/item_to_item.parquet'

# Cargar el archivo Parquet en un DataFrame utilizando PyArrow
table = pq.read_table(file_path)
df = table.to_pandas()


In [83]:
# Reemplazar espacios por comas en la columna top_5_item_id
df['top_5_item_id'] = df['top_5_item_id'].str.replace(' ', ',')

In [84]:
df

,item_id,top_5_item_id
0,761140,"350810,361520,262120,504880,592730"
1,643980,"365270,344190,465830,465831,465832"
2,670290,"522660,729580,478034,486190,485143"
3,767400,"764280,19080,60340,227380,237630"
4,772540,"289480,297100,318220,364800,364810"
...,...,...
22522,745400,"244810,257750,238460,277510,297370"
22523,773640,"901776,80202,80204,80207,238930"
22524,733530,"206690,207710,108232,108230,108200"
22525,610660,"575400,361840,760180,509130,507070"


In [85]:
# Creamos una funcion auxiliar para obtener los primeros 5 item_id
def get_top_5_item_id(item_id):
    row = df[df['item_id'] == item_id]
    if len(row) == 0:
        return None  # Si el item_id no está en el DataFrame, retorna None
    
    top_5 = row['top_5_item_id'].iloc[0].split()[:5]  # Obtiene los primeros 5 item_id de la fila
    return top_5

get_top_5_item_id("761140")


['350810,361520,262120,504880,592730']

In [86]:
# Nos conviene utilizar un df para obtener el title del item_id correspondiente

# Ruta de los archivos Parquet
games_parquet = 'datasets/processed_data/steam_games.parquet'
# Utiliza PyArrow para leer el archivo Parquet
tabla_games = pq.read_table(games_parquet)
# Leer las tablas en DataFrames de Pandas
df_steam_games = tabla_games.to_pandas()

title_finder = df_steam_games[['item_id', 'title']]


In [87]:
title_finder


,item_id,title
88310,761140,Lost Summoner Kitty
88311,643980,Ironbound
88312,670290,Real Pool 3D - Poolians
88313,767400,弹炸人2222
88315,772540,Battle Royale Trainer
...,...,...
120439,745400,Kebab it Up!
120440,773640,Colony On Mars
120441,733530,LOGistICAL: South Africa
120442,610660,Russian Roads


In [95]:
def get_top_5_item_id_with_titles(item_id):
    row = df[df['item_id'] == item_id]
    if len(row) == 0:
        return None  # Si el item_id no está en el DataFrame, retorna None
    
    top_5_ids = row['top_5_item_id'].iloc[0].split(",")[:5]  # Divide los top_5_item_id y obtiene los primeros 5
    titles = []
    for id in top_5_ids:
        id = id.strip()  # Elimina espacios en blanco alrededor del item_id
        title_row = title_finder[title_finder['item_id'] == id]['title']
        if not title_row.empty:
            titles.append((id, title_row.iloc[0]))
    return titles


In [97]:
get_top_5_item_id_with_titles("761140")

[('350810', 'Pixel Puzzles 2: Anime'),
 ('361520', 'World of Cinema - Directors Cut'),
 ('262120', 'Toy Soldiers: Complete'),
 ('504880', 'Pixel Puzzles 2: RADical ROACH'),
 ('592730', 'Aerial Destruction')]

In [53]:
# Finalmente exportamos nuestros datos a utilizar para la API 
title_finder.to_parquet('datasets/processed_data/data_for_funciones/title_finder.parquet')
df.to_parquet('datasets/processed_data/data_for_funciones/recomendacion_juego.parquet')

# Lo unico que nos queda es adaptar la funcion a la API ...

#### Sistema de recomendación user-item:

- **def recomendacion_usuario( user_id):**

> Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [57]:
# Especifica la ruta del archivo Parquet
ruta_archivo_parquet = 'datasets/processed_data/data_for_ML/ml_user_items.parquet'

# Lee el archivo Parquet en una tabla de PyArrow
tabla_parquet = pq.read_table(ruta_archivo_parquet)

# Convierte la tabla a un DataFrame de pandas si es necesario
ml_user_items = tabla_parquet.to_pandas()


In [58]:
# Normalizar playtime_forever en un rango de 0 a 1
scaler = MinMaxScaler()
# Crear una nueva columna para dicha normalización
ml_user_items['normalized_playtime'] = scaler.fit_transform(ml_user_items[['playtime_forever']])

# Eliminar las columnas playtime_forever
ml_user_items = ml_user_items.drop(['playtime_forever'], axis=1)

# elimnar duplicados
ml_user_items = ml_user_items.drop_duplicates(subset=['item_id', 'user_id'])



In [59]:
ml_user_items 

,user_id,item_id,normalized_playtime
0,syates,1002,0.094044
1,76561197973196808,1002,0.021944
2,76561198007521657,1002,0.037618
3,Blarghargh,1002,0.049373
4,76561197969407671,1002,0.006270
...,...,...,...
675015,Rhuri_AGA,9980,0.094044
675016,romakin,9980,0.007053
675017,TandyTandon,9980,0.015674
675018,2cq,9980,0.018809


In [63]:
# Especificamos el valor mínimo y el valor máximo de los tiempos y el dataset a utilizar
reader = Reader(rating_scale=(ml_user_items["normalized_playtime"].min(),ml_user_items["normalized_playtime"].max()))

# El objeto Dataset de Surprise nos permite leer datos 
dataset = Dataset.load_from_df(ml_user_items,reader)

# Empleamos GridSearchCV con el algoritmo SVD y seteamos el parámetro refit a True con measures = ["rmse","fcp"]
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}

gs = GridSearchCV(SVD, param_grid, measures=['fcp',"rmse"], cv=3, refit=True)

# Entrenamos el modelo
gs.fit(dataset)

In [64]:
# Mejor combinacion de parametros
gs.best_params

{'fcp': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6},
 'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}

In [65]:
# Mejor Score
gs.best_score

{'fcp': 0.588281208478125, 'rmse': 0.18666702348096634}

In [66]:
# Guardamos el modelo con mayor fcp
best_model = gs.best_estimator["fcp"]


In [67]:
# Prediccion para user_id : syates,  item_id : 1002 (probamos el modelo)
pred = best_model.predict("syates", "1002")
pred.est

0.06414271750674075

In [68]:
# Definir el user_id para el cual queremos hacer recomendaciones
user_id = "syates"

# Obtener todos los item_id en el dataset
all_item_ids = ml_user_items["item_id"].unique()

# Obtener las predicciones para todos los item_id y seleccionar los 5 con la puntuación más alta
predictions = [(item_id, best_model.predict(user_id, item_id).est) for item_id in all_item_ids]
top_recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]

# Imprimir las recomendaciones
print(f"Recomendaciones para el usuario {user_id}:")
for item_id, score in top_recommendations:
    print(f"Item ID: {item_id}, Score: {score}")



Recomendaciones para el usuario syates:
Item ID: 311560, Score: 0.3070807243356598
Item ID: 349040, Score: 0.2938624710973123
Item ID: 223100, Score: 0.28669939337377054
Item ID: 252450, Score: 0.2865998862727887
Item ID: 348470, Score: 0.28448476032609693


In [69]:
# Lista para almacenar las predicciones
all_predictions = []

# Iterar sobre todos los user_id
for user_id in ml_user_items["user_id"].unique():
    # Calcular las predicciones para todos los item_id
    predictions = [(item_id, best_model.predict(user_id, item_id).est) for item_id in all_item_ids]
    # Seleccionar los 5 item_id con las puntuaciones más altas
    top_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]
    # Guardar los resultados en la lista
    all_predictions.append({"user_id": user_id, "top_5_item_id": [item_id for item_id, _ in top_predictions]})

# Convertir la lista de diccionarios en un DataFrame
predictions_df = pd.DataFrame(all_predictions)



In [70]:
predictions_df

,user_id,top_5_item_id
0,syates,"[311560, 349040, 223100, 252450, 348470]"
1,76561197973196808,"[421050, 311560, 322520, 252450, 495700]"
2,76561198007521657,"[311560, 349040, 223100, 348470, 421050]"
3,Blarghargh,"[311560, 349040, 223100, 348470, 252450]"
4,76561197969407671,"[311560, 349040, 252450, 223100, 346940]"
...,...,...
46332,76561198024357966,"[385690, 404100, 442140, 447780, 34640]"
46333,76561198068756102,"[476920, 495700, 421030, 388540, 403940]"
46334,76561198076743060,"[522990, 487530, 449500, 453390, 437170]"
46335,Joshugawa,"[399240, 327940, 428240, 408640, 500590]"


In [72]:
def user_recomend(user_id):
    row = predictions_df[predictions_df['user_id'] == user_id]
    if not row.empty:
        return row['top_5_item_id'].iloc[0]
    else:
        return "User ID not found"


In [90]:
# Ejemplo de uso:
user_recomend("syates")

['311560', '349040', '223100', '252450', '348470']

In [91]:
# Abrimos el title_finder
ruta_archivo_parquet = 'datasets/processed_data/data_for_funciones/title_finder.parquet'
tabla_parquet = pq.read_table(ruta_archivo_parquet)
title_finder = tabla_parquet.to_pandas()


In [92]:
def user_recommend_with_titles(user_id):
    row = predictions_df[predictions_df['user_id'] == user_id]
    if not row.empty:
        top_5_ids = row['top_5_item_id'].iloc[0][:5]  # Obtenemos los primeros 5 elementos de la lista
        titles = []
        for id in top_5_ids:
            id = id.strip()  # Elimina espacios en blanco alrededor del item_id
            title_row = title_finder[title_finder['item_id'] == id]['title']
            if not title_row.empty:
                titles.append(title_row.iloc[0])
        return titles
    else:
        return "User ID not found"


In [93]:
user_recommend_with_titles("syates")

['Assassin’s Creed® Rogue',
 'NARUTO SHIPPUDEN: Ultimate Ninja STORM 4',
 'Homefront®: The Revolution',
 'StarDrive 2',
 'SAMURAI WARRIORS 4-II']

In [81]:
# exportamos los datos para la funcion en la API
predictions_df.to_parquet('datasets/processed_data/data_for_funciones/recomendacion_usuario.parquet')
predictions_df.to_csv('datasets/processed_data/data_for_funciones/recomendacion_usuario.csv')
